In [ ]:
# Instalando as bibliotecas necessárias
!pip install biopython pandas requests

# Importando os módulos necessários
from tkinter import filedialog,messagebox
from functions import *
from Bio import Entrez,Phylo
import tkinter as tk
import pandas as pd
import requests
import logging

import io


In [ ]:
# Trocar scientificName na coluna
# Configurando o sistema de registro para gravar mensagens em um arquivo de log
# caminho_logfile = salvar_arquivo('.log')
logging.basicConfig(filename='logfile.log', level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')
configure_entrez()

In [ ]:
# Lê o arquivo CSV e armazena os dados em um DataFrame
df = ler_arquivo_csv()

if not df.empty:
    # Copiando os dados de df_scientificname para tax_hierar
    df_scientificname = df.drop_duplicates(subset='scientificName')['scientificName']
    df_scientificname = pd.DataFrame(df_scientificname)
    tax_hierar = df_scientificname.copy()
else:
    # Criando um DataFrame vazio
    tax_hierar = pd.DataFrame(columns=['taxonID', 'taxon', 'ID_pai', 'parent_taxon_index', 
                                       'ID_filho', 'index_filho', 'scientificName_correct', 'scientificName'])


In [ ]:

# Adicionando colunas vazias para armazenar informações de taxonomia
tax_hierar['taxonID'] = None
tax_hierar['taxon'] = None
tax_hierar['ID_pai'] = None
tax_hierar['parent_taxon_index'] = None
tax_hierar['ID_filho'] = None
tax_hierar['index_filho'] = None
tax_hierar['scientificName_correct'] = None

# Dividindo a coluna 'scientificName' em listas separadas por ' | '
tax_hierar['scientificName'] = tax_hierar['scientificName'].str.split(' \| ')

# Expandindo as listas para linhas individuais no DataFrame
tax_hierar = tax_hierar.explode('scientificName')

# Removendo entradas duplicadas de 'scientificName'
tax_hierar.drop_duplicates(subset='scientificName', inplace=True, ignore_index=True)

# Adicionando mensagem de log para o término da operação
logging.info("Preparação do DataFrame 'tax_hierar' concluída com sucesso.")


In [ ]:

# Lista para armazenar nomes de espécies que não puderam ser encontradas na pesquisa de taxonomia
lista_erro = []

# Iterando sobre os valores da coluna 'scientificName' do DataFrame 'tax_hierar'
for i, valor in enumerate(tax_hierar['scientificName']):
    # Chamando a função 'get_taxonomy_id' para obter informações de taxonomia para o valor atual
    tax_hierar.loc[i, 'scientificName_correct'], tax_hierar.loc[i, 'taxonID'], tax_hierar.loc[i, 'taxon'], tax_hierar.loc[i, 'ID_pai'], erro = get_taxonomy_id(valor)

    # Verificando se ocorreu um erro durante a busca de taxonomia
    if erro:
        lista_erro.append(erro)
        logging.warning(f"Erro ao obter informações de taxonomia para '{valor}': Nome não encontrado na pesquisa.")

tax_hierar_taxonid_none = tax_hierar[tax_hierar['taxonID'].isna()]

# Removendo linhas onde 'taxonID' está vazio, indicando que a busca de taxonomia não teve êxito
tax_hierar.dropna(subset='taxonID',inplace=True)
logging.info("Informações de taxonomia obtidas com sucesso para todas as espécies.")

# Resetando o índice do DataFrame após a remoção de linhas
tax_hierar.reset_index(drop=True, inplace=True)

# Adicionando mensagem de log para o término da operação
logging.info("Obtenção de informações de taxonomia para o DataFrame 'tax_hierar' concluída com sucesso.")


In [ ]:
# Lista temporária para armazenar informações de taxonomia de espécies encontradas
temp_df = []

# Iterando sobre os valores da coluna 'taxonID' do DataFrame 'tax_hierar'
for i, valor in enumerate(tax_hierar['taxonID']):
    # Parâmetros para a solicitação HTTP
    parametros = {'txid': valor, 'rank': 'custom', 'srank': rank_dwc, 'format': 'json'}

    # Realizando uma solicitação GET para a API de taxonomia
    response = requests.get(url_api, params=parametros)
    # Verificando se a resposta foi bem-sucedida
    if response.status_code == 200:
        data = response.json()
        dicta = list(data.values())[0]

        # Iterando sobre os itens retornados pela API
        for chave, valor in dicta.items():
            # Verificando se o valor não contém '_' e se não existe no 'temp_df' ou em 'tax_hierar'
            if '_' not in valor and not any(entry['scientificName'] == valor for entry in temp_df) and valor not in tax_hierar['scientificName'].tolist():
                # Chamando a função 'get_taxonomy_id' para obter informações de taxonomia para o valor atual
                temp_name, temp_txid, temp_rank, temp_pai, erro = get_taxonomy_id(valor)

                # Verificando se ocorreu um erro durante a busca de taxonomia
                if erro:
                    lista_erro.append(erro)
                    logging.warning(f"Erro ao obter informações de taxonomia para '{valor}': {erro}")

                # Adicionando as informações obtidas ao 'temp_df'
                temp_df.append({'scientificName': valor, 'ID_filho': None, 'ID_pai': temp_pai, 'taxonID': temp_txid, 'taxon': temp_rank, 'scientificName_correct': temp_name})
    else:
        logging.error(f"Falha ao acessar a API de taxonomia para o taxonID '{valor}'. Status code: {response.status_code}")

# Concatenando 'tax_hierar' com o 'temp_df' e removendo linhas onde 'taxonID' está vazio
tax_hierar = pd.concat([tax_hierar, pd.DataFrame(temp_df)])
tax_hierar.drop(tax_hierar.loc[tax_hierar['taxonID'] == ''].index, inplace=True)

# Removendo entradas duplicadas com base em 'taxonID' e redefinindo o índice
tax_hierar.drop_duplicates(subset=['taxonID'], inplace=True, ignore_index=True)
tax_hierar.reset_index(drop=True, inplace=True)

logging.info("Concluída a busca e concatenação de informações de taxonomia.")


In [ ]:
# Removendo itens duplicados e vazios da lista de erros
lista_erro = [item for item in list(set(lista_erro)) if item and item not in tax_hierar['scientificName']]

# Lista temporária para armazenar informações de taxonomia de espécies encontradas com erro
temp_df_erro = []

# Loop enquanto houver itens na lista de erros
while len(lista_erro) > 0:
    # Verificando se o primeiro item da lista de erros não está em 'tax_hierar'
    if lista_erro[0] not in tax_hierar['scientificName']:
        # Chamando a função 'get_taxonomy_id' para obter informações de taxonomia para o valor atual
        temp_name, temp_txid, temp_rank, temp_pai, erro = get_taxonomy_id(valor)

        # Verificando se ocorreu um erro durante a busca de taxonomia
        if erro:
            lista_erro.append(erro)

        # Adicionando as informações obtidas ao 'temp_df_erro'
        temp_df_erro.append({'scientificName': valor, 'ID_filho': None, 'ID_pai': temp_pai, 'taxonID': temp_txid, 'taxon': temp_rank, 'scientificName_correct': temp_name})
    lista_erro.pop(0)

# Concatenando 'tax_hierar' com o 'temp_df_erro' e removendo linhas onde 'taxonID' está vazio
tax_hierar = pd.concat([tax_hierar, pd.DataFrame(temp_df_erro)])
tax_hierar.drop(tax_hierar.loc[tax_hierar['taxonID'] == ''].index, inplace=True)

# Removendo entradas duplicadas com base em 'taxonID' e redefinindo o índice
tax_hierar.drop_duplicates(subset=['taxonID'], inplace=True, ignore_index=True)
tax_hierar.reset_index(drop=True, inplace=True)

logging.info("Concluída a busca e concatenação de informações de taxonomia com tratamento de erros.")


In [ ]:
# Dicionário para mapear o ID do pai para o índice correspondente no DataFrame 'tax_hierar'
pai_to_index = {value: index for index, value in enumerate(tax_hierar['taxonID'].unique())}

# Iterando sobre os valores da coluna 'ID_pai' do DataFrame 'tax_hierar'
for i, valor in enumerate(tax_hierar['ID_pai']):
    # Número máximo de tentativas para obter informações de taxonomia
    num_tentativas = 3
    tentativa = 0

    # Verificando se o valor está presente na coluna 'taxonID'
    if valor in tax_hierar['taxonID'].values:
        # Obtendo o índice do pai e atribuindo ao 'parent_taxon_index'
        tax_hierar.loc[i, 'parent_taxon_index'] = tax_hierar.index[tax_hierar['taxonID'] == valor].tolist()[0]
    else:
        # Tentativas para encontrar o pai na hierarquia taxonômica
        while tentativa < num_tentativas:
            # Se o taxon atual for 'kingdom', interrompe a tentativa
            if tax_hierar.loc[i, 'taxon'] == 'kingdom':
                break

            try:
                # Obtendo informações de taxonomia para o ID do pai
                record = efetch_NCBI(str(tax_hierar.loc[i, 'ID_pai']))

                # Obtendo o ID do pai do pai
                taxid_sup_rank = record[0]['LineageEx'][-1]['TaxId']
                tax_hierar.loc[i, 'ID_pai'] = taxid_sup_rank

                # Se o rank do pai for 'kingdom', interrompe a tentativa
                if record[0]['Rank'] == 'kingdom':
                    break

                # Se o ID do pai estiver mapeado, atribui o índice ao 'parent_taxon_index'
                if taxid_sup_rank in pai_to_index:
                    tax_hierar.loc[i, 'parent_taxon_index'] = pai_to_index[taxid_sup_rank]
                    break

            except Exception as e:
                # Se ocorrer um erro, imprime o erro e tenta novamente
                logging.error(f"Erro na tentativa {tentativa + 1}: {str(e)} {valor}")
                tentativa += 1

# Substituindo valores 'None' por 'NaN' na coluna 'parent_taxon_index'
tax_hierar['parent_taxon_index'].replace({None: 'NaN'}, inplace=True)

logging.info("Concluída a atribuição de índices de pais para cada espécie na hierarquia taxonômica.")


In [ ]:
# Agrupando os índices pelo índice do pai
a = tax_hierar.groupby(['parent_taxon_index'], group_keys=True).groups

# Iterando sobre os grupos e seus valores
for chave, valor in a.items():
    # Verificando se a chave está presente nos índices do DataFrame
    if chave in tax_hierar.index:
        # Convertendo os valores para uma string separada por vírgulas e atribuindo ao 'index_filho'
        tax_hierar.loc[chave, 'index_filho'] = ','.join(map(str, valor.tolist()))

        # Obtendo os taxonIDs dos filhos e atribuindo ao 'ID_filho'
        l = list(tax_hierar.loc[valor, 'taxonID'])
        tax_hierar.loc[chave, 'ID_filho'] = ','.join(l)

logging.info("Concluída a atribuição de índices de filhos para cada espécie na hierarquia taxonômica.")


In [ ]:
# Renomeando as colunas 'scientificName' e 'scientificName_correct'
tax_hierar = tax_hierar.rename(columns={'scientificName':'scientificName_search', 'scientificName_correct':'scientificName'})

arquivo_salvar = 'scientificName1.csv'#salvar_arquivo()
if arquivo_salvar:
    # Salvando o DataFrame em um arquivo CSV
    tax_hierar.to_csv(arquivo_salvar, index=False)

    logging.info(f"Salvo o DataFrame em '{arquivo_salvar}' com sucesso.")

else:
    logging.info("Operação de salvamento de arquivo não foi completada.")


In [ ]:
root = tax_hierar[tax_hierar['taxon'] == 'kingdom']

arvore_taxonomica = construir_arvore_taxonomica(tax_hierar, 'scientificName', 'index_filho')
arvore_newick = arvore_para_newick(arvore_taxonomica, 'Eukaryota')
tree = Phylo.read(io.StringIO(arvore_newick), "newick")

# Visualizando a árvore
Phylo.draw(tree)
# Salvar a árvore em formato Newick
Phylo.write(tree, "arvore/arvore_taxonomica.nwk", "newick")
# Salvar a árvore em formato Nexus
Phylo.write(tree, "arvore/arvore_taxonomica.nex", "nexus")
# Salvar a árvore em formato NeXML
Phylo.write(tree, "arvore/arvore_taxonomica.xml", "nexml")
